The active/lazy training paper does not attempt to trigger active/lazy regime via initialization. Instead, the paper applies a scaling factor alpha to the loss function. \\

Efforts following this approach proved to be more fruitful than my earlier attempts to observe active/lazy training via controlling the variance of initialization values. 
I have used the scaling method from the paper’s CIFAR10 CNN experiment in a single-hidden-layer neural network. The general observation remains the same:
1. Experiment shows that the l2-norm of weight movements away from initialization values increase with scaling alpha, which is consistent with theoretical assumptions.
2. I have observed that there is a threshold for alpha at which movements of weight begin to occur. Scaling factors smaller than this threshold would see little/zero weight movement whereas larger scaling factors lead to increasing movements past this threshold.
3. With respect to decreasing scaling factor alpha while holding other factors constant, model accuracy peaks at a point after the threshold at which weights begin to move (not at the threshold). After this peak, model accuracy would then deteriorate to that of random selection (both training and dev set to around 0.1 for CIFAR10). Experimental results also show that as the scaling factor continues to increase, weight movements from initialization experiences a temporary drop after the performance peak before resuming to increase monotonously at increasing rates.
4. Learning rate and (likely) model size and architecture have influence on not only model performance, but also where the active/lazy threshold occurs: experimental result shows that 
larger models see later performance peak (wrt. decreasing alpha), slower performance deterioration, and slower increase in weight movements. 
holding other factors constant, decreasing learning rate led to:
Smaller active/lazy threshold alpha (scaling factor alpha at which weights begin to move during training)
Slower increase in weight movements as alpha decreases
Later performance peak as alpha decreases
Slower performance deterioration as alpha decreases
holding other factors constant, increasing hidden layer width led to:
Unsurprisingly, better accuracy
Faster increase in weight movements as alpha decreases
5. While the paper’s code produced reasonable accuracies at large alpha values (lazy regime), my two-layer nn experiments show poor model performance (both training and dev set are at the random selection accuracy around 0.1) under the lazy training regime. This difference can be caused by major differences in model architectures (the paper implements a variation of VGG ConvNet). \\

Next step is to integrate into the Convnet architecture used in Sam’s double descent experiments.



In [3]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.layers import Dense, Flatten 
from tensorflow.keras import Model
import matplotlib.pyplot as plt

2.4.1


In [4]:
DATA = 'CIFAR10'
NORMALIZE = True

In [5]:
if DATA == 'MNIST':
  mnist = tf.keras.datasets.mnist
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
if DATA == 'CIFAR10':
  cifar10 = tf.keras.datasets.cifar10
  (x_train, y_train), (x_test, y_test) = cifar10.load_data()
if NORMALIZE:
  x_train, x_test = x_train / 255.0, x_test / 255.0

In [6]:
class CustomCallback_epoch(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.dh = display('SAVING INITIAL WEIGHT VALUES', display_id=True)
        self.start_time = time.perf_counter()

    def on_train_begin(self, logs=None):
        global weight_history
        # print('SAVING INITIAL WEIGHT VALUES')
        weight_history.append(self.model.get_weights())

    def on_epoch_end(self, epoch, logs=None):
        global weight_history
        # list of weight tensors
        curr_weight = self.model.get_weights()
        if weight_history:
          weight_change = [curr_weight[i] - weight_history[0][i] for i in range(len(curr_weight))]
          norm_delta = [tf.norm(t, ord=2).numpy() for t in weight_change]
          # print('L2 NORM OF WEIGHT CHANGE RELATIVE TO INITIAL VALUES: ', norm_delta)
          weight_history.append(norm_delta)

        end_time = time.perf_counter()
        run_time = end_time - self.start_time
        hrs, mnts, secs = int(run_time // 60 // 60), int(run_time // 60 % 60), int(run_time % 60)

        template = 'Epoch: {:04}, Total Run Time: {:02}:{:02}:{:02}'
        template += ' - Loss: {:.4e}, Accuracy: {:.3f}, Test Loss: {:.4e}, Test Accuracy: {:.3f}'
        template += ' - L2 Norm of Weight Movement From Initialization: %s' %str(norm_delta)

        train_loss, train_accuracy = logs['loss'], logs['accuracy']
        test_loss, test_accuracy = logs['val_loss'], logs['val_accuracy']
        self.dh.update(template.format(epoch, hrs, mnts, secs, train_loss, train_accuracy, test_loss, test_accuracy))
  

In [7]:
# some parameters in the active/lazy paper:
# optimizer: sgd with momentum=0.9
# initialization: kernel - xavier(glorot) normal, bias - zeros (default for Dense in tf)
# loss: both categroical crossentropy(sparse) and mse are provided as options, using cce here
# scaling: implemented in the paper repo's train.py as loss = torch.nn.CrossEntropyLoss(alpha*outputs, targets)/alpha**2
#          implemented here with scaled_custom_loss() loss function that does the same thing        

def train(alpha, epoch, opt, lr, scaling=True, width=64):
    global weight_history
    weight_history = []

    def scaled_custom_loss(y_actual, y_pred):
        sce = tf.keras.losses.SparseCategoricalCrossentropy()
        scaled_sce = sce(y_actual, alpha*y_pred)/alpha**2
        return scaled_sce

    if DATA == 'CIFAR10':
      flatten = tf.keras.layers.Flatten(input_shape=(32, 32, 3))

    if DATA == 'MNIST':
      flatten = tf.keras.layers.Flatten(input_shape=(28, 28))

    model = tf.keras.models.Sequential([
      flatten,
      tf.keras.layers.Dense(width, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    # model.summary()
  
    if opt=='adam':
      # default lr for Adam is 0.001
      opt = tf.keras.optimizers.Adam(learning_rate=lr)
    elif opt=='sgd':
      # default lr for sgd is 0.01
      opt = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
    else:
      raise Exception('optimizer must be adam/sgd')

    model.compile(optimizer=opt,
                  loss=scaled_custom_loss if scaling else 'sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train, epochs=epoch, validation_data=(x_test, y_test), callbacks=[CustomCallback_epoch()], verbose=0)
    
    print()
    print('max training accuracy', max(history.history['accuracy']))
    print('min training loss', min(history.history['loss']))
    print('max validation accuracy', max(history.history['val_accuracy']))
    print('min validation loss', min(history.history['val_loss']))

    # plot(history)
    # print(tf.math.confusion_matrix(y_test, tf.argmax(model.predict(x_test), axis=1)))
    print()
    print('l2-normed weight changes from initial values after last epoch:')
    print(weight_history[-1])

    return (weight_history[1:], history.history)

In [20]:
unscaled = train(alpha=None, epoch=10, opt='sgd', lr=0.01, scaling=False)

'Epoch: 0009, Total Run Time: 00:00:45 - Loss: 1.6841e+00, Accuracy: 0.390, Test Loss: 1.7300e+00, Test Accuracy: 0.383 - L2 Norm of Weight Movement From Initialization: [13.935824, 3.1807551, 2.3657033, 2.7277188]'


max training accuracy 0.3919999897480011
min training loss 1.6840555667877197
max validation accuracy 0.40130001306533813
min validation loss 1.6713179349899292

l2-normed weight changes from initial values after last epoch:
[13.935824, 3.1807551, 2.3657033, 2.7277188]


In [13]:
lrs = [1.0, 0.1, 0.01, 0.001]
alphas = [10000000.0, 1000000.0, 100000.0, 10000.0, 1000.0, 100.0, 10.0, 5.0, 1.0, 0.5, 0.1, 0.01]
normed_weight_changes = {}
optimizer = 'sgd'
num_epochs = 10
for learning_rate in lrs:
  for alpha_val in alphas:
    print('='*80)
    print('opt = %s, lr = %f, alpha = %f' %(optimizer, learning_rate, alpha_val))
    print('='*80)
    normed_weight_changes[(optimizer, learning_rate, alpha_val)] = train(alpha=alpha_val, epoch=num_epochs, opt=optimizer, lr=learning_rate, scaling=True)

opt = sgd, lr = 1.000000, alpha = 10000000.000000


'Epoch: 0009, Total Run Time: 00:00:45 - Loss: 2.3026e-14, Accuracy: 0.095, Test Loss: 2.3026e-14, Test Accuracy: 0.099 - L2 Norm of Weight Movement From Initialization: [0.0, 0.0, 0.0, 0.0]'


max training accuracy 0.09533999860286713
min training loss 2.3025665711129753e-14
max validation accuracy 0.09910000115633011
min validation loss 2.3025850364312254e-14

l2-normed weight changes from initial values after last epoch:
[0.0, 0.0, 0.0, 0.0]
opt = sgd, lr = 1.000000, alpha = 1000000.000000


'Epoch: 0009, Total Run Time: 00:00:46 - Loss: 2.3026e-12, Accuracy: 0.106, Test Loss: 2.3026e-12, Test Accuracy: 0.107 - L2 Norm of Weight Movement From Initialization: [0.0, 0.0, 0.0, 0.0]'


max training accuracy 0.10605999827384949
min training loss 2.3025997341469262e-12
max validation accuracy 0.1071000024676323
min validation loss 2.302588675284767e-12

l2-normed weight changes from initial values after last epoch:
[0.0, 0.0, 0.0, 0.0]
opt = sgd, lr = 1.000000, alpha = 100000.000000


...


max training accuracy 0.45451998710632324
min training loss 6.123098850250244
max validation accuracy 0.4447000026702881
min validation loss 6.286983966827393

l2-normed weight changes from initial values after last epoch:
[10.945328, 1.7978578, 2.5358744, 2.3660274]
opt = sgd, lr = 0.001000, alpha = 0.100000


'Epoch: 0009, Total Run Time: 00:00:46 - Loss: 2.3104e+02, Accuracy: 0.099, Test Loss: 2.3058e+02, Test Accuracy: 0.100 - L2 Norm of Weight Movement From Initialization: [25.359215, 0.89406925, 17.915815, 0.25575876]'


max training accuracy 0.10255999863147736
min training loss 230.9173126220703
max validation accuracy 0.10000000149011612
min validation loss 230.57913208007812

l2-normed weight changes from initial values after last epoch:
[25.359215, 0.89406925, 17.915815, 0.25575876]
opt = sgd, lr = 0.001000, alpha = 0.010000


'Epoch: 0009, Total Run Time: 00:00:45 - Loss: 1.0362e+05, Accuracy: 0.100, Test Loss: 1.0362e+05, Test Accuracy: 0.100 - L2 Norm of Weight Movement From Initialization: [524.8614, 22.775066, 73.9547, 19.422792]'


max training accuracy 0.10000000149011612
min training loss 103571.109375
max validation accuracy 0.10000000149011612
min validation loss 103617.0703125

l2-normed weight changes from initial values after last epoch:
[524.8614, 22.775066, 73.9547, 19.422792]


In [8]:
lrs = [1.0, 0.1, 0.01, 0.001]
alphas = [20000.0, 1000.0, 100.0, 80.0, 60.0, 40.0, 30.0, 20.0, 10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0, 0.8, 0.6, 0.4, 0.2, 0.1, 0.07, 0.04, 0.01, 0.001]
normed_weight_changes = {}
optimizer = 'sgd'
num_epochs = 10
for learning_rate in lrs:
  for alpha_val in alphas:
    print('='*80)
    print('opt = %s, lr = %f, alpha = %f' %(optimizer, learning_rate, alpha_val))
    print('='*80)
    normed_weight_changes[(optimizer, learning_rate, alpha_val)] = train(alpha=alpha_val, epoch=num_epochs, opt=optimizer, lr=learning_rate, scaling=True)

opt = sgd, lr = 1.000000, alpha = 20000.000000


'Epoch: 0009, Total Run Time: 00:01:01 - Loss: 5.7565e-09, Accuracy: 0.106, Test Loss: 5.7564e-09, Test Accuracy: 0.102 - L2 Norm of Weight Movement From Initialization: [0.0, 0.0, 0.0, 0.0]'


max training accuracy 0.10608000308275223
min training loss 5.756487286845413e-09
max validation accuracy 0.10189999639987946
min validation loss 5.7564477629057365e-09

l2-normed weight changes from initial values after last epoch:
[0.0, 0.0, 0.0, 0.0]
opt = sgd, lr = 1.000000, alpha = 1000.000000


'Epoch: 0009, Total Run Time: 00:01:03 - Loss: 2.3026e-06, Accuracy: 0.113, Test Loss: 2.3026e-06, Test Accuracy: 0.115 - L2 Norm of Weight Movement From Initialization: [0.0, 0.0, 0.0, 0.0]'


max training accuracy 0.11259999871253967
min training loss 2.302632083228673e-06
max validation accuracy 0.1151999980211258
min validation loss 2.3025800146569964e-06

l2-normed weight changes from initial values after last epoch:
[0.0, 0.0, 0.0, 0.0]
opt = sgd, lr = 1.000000, alpha = 100.000000


...


max training accuracy 0.10010000318288803
min training loss 7251.66259765625
max validation accuracy 0.10000000149011612
min validation loss 7255.8232421875

l2-normed weight changes from initial values after last epoch:
[29.170166, 1.0205396, 27.17494, 1.0541494]
opt = sgd, lr = 0.001000, alpha = 0.010000


'Epoch: 0009, Total Run Time: 00:00:47 - Loss: 1.0362e+05, Accuracy: 0.100, Test Loss: 1.0362e+05, Test Accuracy: 0.100 - L2 Norm of Weight Movement From Initialization: [526.5488, 20.098198, 72.01266, 23.057837]'


max training accuracy 0.10000000149011612
min training loss 103571.25
max validation accuracy 0.10000000149011612
min validation loss 103617.0546875

l2-normed weight changes from initial values after last epoch:
[526.5488, 20.098198, 72.01266, 23.057837]
opt = sgd, lr = 0.001000, alpha = 0.001000


'Epoch: 0009, Total Run Time: 00:00:48 - Loss: 8.2902e+06, Accuracy: 0.100, Test Loss: 8.2902e+06, Test Accuracy: 0.100 - L2 Norm of Weight Movement From Initialization: [83535.76, 2984.2903, 11637.054, 3040.1663]'


max training accuracy 0.10000000149011612
min training loss 8286862.0
max validation accuracy 0.10000000149011612
min validation loss 8290207.5

l2-normed weight changes from initial values after last epoch:
[83535.76, 2984.2903, 11637.054, 3040.1663]


In [6]:
# experiment with wider network
lrs = [1.0, 0.1, 0.01, 0.001]
alphas = [10000000.0, 1000000.0, 100000.0, 10000.0, 1000.0, 100.0, 10.0, 5.0, 1.0, 0.5, 0.1, 0.01]
normed_weight_changes_w256 = {}
optimizer = 'sgd'
num_epochs = 10
for learning_rate in lrs:
  for alpha_val in alphas:
    print('='*80)
    print('opt = %s, lr = %f, alpha = %f' %(optimizer, learning_rate, alpha_val))
    print('='*80)
    normed_weight_changes_w256[(optimizer, learning_rate, alpha_val)] = train(alpha=alpha_val, epoch=num_epochs, opt=optimizer, lr=learning_rate, scaling=True, width=256)

opt = sgd, lr = 1.000000, alpha = 10000000.000000


'Epoch: 0009, Total Run Time: 00:01:31 - Loss: 2.3026e-14, Accuracy: 0.094, Test Loss: 2.3026e-14, Test Accuracy: 0.093 - L2 Norm of Weight Movement From Initialization: [0.0, 0.0, 0.0, 0.0]'


max training accuracy 0.09350000321865082
min training loss 2.3025665711129753e-14
max validation accuracy 0.09279999881982803
min validation loss 2.3025850364312254e-14

l2-normed weight changes from initial values after last epoch:
[0.0, 0.0, 0.0, 0.0]
opt = sgd, lr = 1.000000, alpha = 1000000.000000


'Epoch: 0009, Total Run Time: 00:01:33 - Loss: 2.3026e-12, Accuracy: 0.086, Test Loss: 2.3026e-12, Test Accuracy: 0.087 - L2 Norm of Weight Movement From Initialization: [0.0, 0.0, 0.0, 0.0]'


max training accuracy 0.08643999695777893
min training loss 2.3025997341469262e-12
max validation accuracy 0.08709999918937683
min validation loss 2.302588675284767e-12

l2-normed weight changes from initial values after last epoch:
[0.0, 0.0, 0.0, 0.0]
opt = sgd, lr = 1.000000, alpha = 100000.000000


...


max training accuracy 0.5079799890518188
min training loss 5.557001113891602
max validation accuracy 0.4821999967098236
min validation loss 5.8831682205200195

l2-normed weight changes from initial values after last epoch:
[13.602513, 1.6406225, 3.7420678, 2.4693606]
opt = sgd, lr = 0.001000, alpha = 0.100000


'Epoch: 0009, Total Run Time: 00:01:30 - Loss: 1.2434e+03, Accuracy: 0.100, Test Loss: 1.2434e+03, Test Accuracy: 0.100 - L2 Norm of Weight Movement From Initialization: [18.55185, 0.7219624, 19.908718, 0.798692]'


max training accuracy 0.10007999837398529
min training loss 1241.932373046875
max validation accuracy 0.10000000149011612
min validation loss 1243.396484375

l2-normed weight changes from initial values after last epoch:
[18.55185, 0.7219624, 19.908718, 0.798692]
opt = sgd, lr = 0.001000, alpha = 0.010000


'Epoch: 0009, Total Run Time: 00:01:30 - Loss: 1.0362e+05, Accuracy: 0.100, Test Loss: 1.0362e+05, Test Accuracy: 0.100 - L2 Norm of Weight Movement From Initialization: [739.8659, 26.920643, 214.2995, 29.101757]'


max training accuracy 0.10000000149011612
min training loss 103573.1171875
max validation accuracy 0.10000000149011612
min validation loss 103617.0703125

l2-normed weight changes from initial values after last epoch:
[739.8659, 26.920643, 214.2995, 29.101757]


In [9]:
lrs = [1.0, 0.1, 0.01, 0.001]
alphas = [20000.0, 1000.0, 100.0, 80.0, 60.0, 40.0, 30.0, 20.0, 10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0, 0.8, 0.6, 0.4, 0.2, 0.1, 0.07, 0.04, 0.01, 0.001]
normed_weight_changes_w256 = {}
optimizer = 'sgd'
num_epochs = 10
for learning_rate in lrs:
  for alpha_val in alphas:
    print('='*80)
    print('opt = %s, lr = %f, alpha = %f' %(optimizer, learning_rate, alpha_val))
    print('='*80)
    normed_weight_changes_w256[(optimizer, learning_rate, alpha_val)] = train(alpha=alpha_val, epoch=num_epochs, opt=optimizer, lr=learning_rate, scaling=True, width=256)

opt = sgd, lr = 1.000000, alpha = 20000.000000


'Epoch: 0009, Total Run Time: 00:01:29 - Loss: 5.7565e-09, Accuracy: 0.097, Test Loss: 5.7564e-09, Test Accuracy: 0.098 - L2 Norm of Weight Movement From Initialization: [0.0, 0.0, 0.0, 0.0]'


max training accuracy 0.09724000096321106
min training loss 5.756487286845413e-09
max validation accuracy 0.09769999980926514
min validation loss 5.7564477629057365e-09

l2-normed weight changes from initial values after last epoch:
[0.0, 0.0, 0.0, 0.0]
opt = sgd, lr = 1.000000, alpha = 1000.000000


'Epoch: 0009, Total Run Time: 00:01:30 - Loss: 2.3026e-06, Accuracy: 0.105, Test Loss: 2.3026e-06, Test Accuracy: 0.106 - L2 Norm of Weight Movement From Initialization: [0.0, 0.0, 0.0, 0.0]'


max training accuracy 0.10543999820947647
min training loss 2.302632083228673e-06
max validation accuracy 0.10599999874830246
min validation loss 2.3025800146569964e-06

l2-normed weight changes from initial values after last epoch:
[0.0, 0.0, 0.0, 0.0]
opt = sgd, lr = 1.000000, alpha = 100.000000


...


max training accuracy 0.10000000149011612
min training loss 7252.4599609375
max validation accuracy 0.10000000149011612
min validation loss 7255.8232421875

l2-normed weight changes from initial values after last epoch:
[38.611435, 1.405106, 56.614784, 1.6976434]
opt = sgd, lr = 0.001000, alpha = 0.010000


'Epoch: 0009, Total Run Time: 00:01:27 - Loss: 1.0362e+05, Accuracy: 0.100, Test Loss: 1.0362e+05, Test Accuracy: 0.100 - L2 Norm of Weight Movement From Initialization: [548.04144, 12.98345, 144.25642, 12.287204]'


max training accuracy 0.10000000149011612
min training loss 103574.078125
max validation accuracy 0.10000000149011612
min validation loss 103617.0703125

l2-normed weight changes from initial values after last epoch:
[548.04144, 12.98345, 144.25642, 12.287204]
opt = sgd, lr = 0.001000, alpha = 0.001000


'Epoch: 0009, Total Run Time: 00:01:29 - Loss: 8.2902e+06, Accuracy: 0.100, Test Loss: 8.2902e+06, Test Accuracy: 0.100 - L2 Norm of Weight Movement From Initialization: [58834.17, 2367.6875, 19716.24, 2814.171]'


max training accuracy 0.10000000149011612
min training loss 8287156.5
max validation accuracy 0.10000000149011612
min validation loss 8290206.5

l2-normed weight changes from initial values after last epoch:
[58834.17, 2367.6875, 19716.24, 2814.171]


In [ ]:
# experiment with default network size (width=64) but using adam optimizer
lrs = [1.0, 0.1, 0.01, 0.001, 0.0001]
alphas = [20000.0, 1000.0, 100.0, 80.0, 60.0, 40.0, 30.0, 20.0, 10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0, 0.8, 0.6, 0.4, 0.2, 0.1, 0.07, 0.04, 0.01, 0.001]
normed_weight_changes_adam = {}
optimizer = 'adam'
num_epochs = 10
for learning_rate in lrs:
  for alpha_val in alphas:
    print('='*80)
    print('opt = %s, lr = %f, alpha = %f' %(optimizer, learning_rate, alpha_val))
    print('='*80)
    normed_weight_changes_adam[(optimizer, learning_rate, alpha_val)] = train(alpha=alpha_val, epoch=num_epochs, opt=optimizer, lr=learning_rate, scaling=True)

opt = adam, lr = 1.000000, alpha = 10000000.000000


'Epoch: 0009, Total Run Time: 00:00:48 - Loss: 2.3026e-14, Accuracy: 0.090, Test Loss: 2.3026e-14, Test Accuracy: 0.097 - L2 Norm of Weight Movement From Initialization: [0.0, 0.0, 0.0, 0.0]'


max training accuracy 0.09009999781847
min training loss 2.3025665711129753e-14
max validation accuracy 0.09709999710321426
min validation loss 2.3025850364312254e-14

l2-normed weight changes from initial values after last epoch:
[0.0, 0.0, 0.0, 0.0]
opt = adam, lr = 1.000000, alpha = 1000000.000000


'Epoch: 0009, Total Run Time: 00:00:53 - Loss: 2.3026e-12, Accuracy: 0.094, Test Loss: 2.3026e-12, Test Accuracy: 0.096 - L2 Norm of Weight Movement From Initialization: [0.0, 0.0, 0.0, 0.0]'


max training accuracy 0.09443999826908112
min training loss 2.3025997341469262e-12
max validation accuracy 0.09600000083446503
min validation loss 2.302588675284767e-12

l2-normed weight changes from initial values after last epoch:
[0.0, 0.0, 0.0, 0.0]
opt = adam, lr = 1.000000, alpha = 100000.000000


...

'Epoch: 0009, Total Run Time: 00:01:01 - Loss: 1.2434e+03, Accuracy: 0.100, Test Loss: 1.2434e+03, Test Accuracy: 0.100 - L2 Norm of Weight Movement From Initialization: [2353.5745, 42.46457, 134.58029, 18.991018]'


max training accuracy 0.10000000149011612
min training loss 1243.297607421875
max validation accuracy 0.10000000149011612
min validation loss 1243.39697265625

l2-normed weight changes from initial values after last epoch:
[2353.5745, 42.46457, 134.58029, 18.991018]
opt = adam, lr = 1.000000, alpha = 0.010000


'Epoch: 0009, Total Run Time: 00:01:08 - Loss: 1.0362e+05, Accuracy: 0.100, Test Loss: 1.0362e+05, Test Accuracy: 0.100 - L2 Norm of Weight Movement From Initialization: [2400.2646, 43.306286, 136.94647, 18.991064]'


max training accuracy 0.10000000149011612
min training loss 103570.265625
max validation accuracy 0.10000000149011612
min validation loss 103617.0703125

l2-normed weight changes from initial values after last epoch:
[2400.2646, 43.306286, 136.94647, 18.991064]
opt = adam, lr = 0.100000, alpha = 10000000.000000


'SAVING INITIAL WEIGHT VALUES'

In [ ]:
# experiment with default network size (width=64) but more epochs (100)
lrs = [1.0, 0.1, 0.01, 0.001]
alphas = [20000.0, 1000.0, 100.0, 80.0, 60.0, 40.0, 30.0, 20.0, 10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0, 0.8, 0.6, 0.4, 0.2, 0.1, 0.07, 0.04, 0.01, 0.001]
normed_weight_changes_e100 = {}
optimizer = 'sgd'
num_epochs = 100
for learning_rate in lrs:
  for alpha_val in alphas:
    print('='*80)
    print('opt = %s, lr = %f, alpha = %f' %(optimizer, learning_rate, alpha_val))
    print('='*80)
    normed_weight_changes_e100[(optimizer, learning_rate, alpha_val)] = train(alpha=alpha_val, epoch=num_epochs, opt=optimizer, lr=learning_rate, scaling=True)

In [ ]:
for i in range(20):
  time.sleep(3600)